In [1]:
##Arcgis License
import arcpy
from arcpy.sa import *
from Solar_Calculation import *

In [ ]:
## set workspace (need to change)
arcpy.env.workspace = 'data'
arcpy.env.overwriteOutput = True
##load extensions
arcpy.CheckOutExtension("3D")
arcpy.CheckOutExtension("Spatial")

In [ ]:
def arc_beg(work_path):
    """This function is used to set workspace and load extensions of Arcgis. The input should be the path all files store."""
    arcpy.env.workspace = work_path  # set workspace
    arcpy.env.overwriteOutput = True
    arcpy.CheckOutExtension("3D")  # load 3D Analyst tool
    arcpy.CheckOutExtension("Spatial")  # load Spatial Analyst tool
    print("The workspace has set to: " + work_path)
    return

In [2]:
arc_beg('data/temporary data/')

The workspace has set to: data/temporary data/


In [ ]:
#Create LAS Dataset (3D Analyst or Spatial Analyst)
# input can be a folder if choose recursion
arcpy.CreateLasDataset_management('data/Government Hill/1662_2638.las', 'GH.lasd', 'RECURSION')

In [ ]:
#LAS Dataset to Raster (Spatial Analyst or 3D Analyst)
arcpy.LasDatasetToRaster_conversion('GH.lasd', 'GH', "ELEVATION", "BINNING AVERAGE NATURAL_NEIGHBOR",
                                   "FLOAT", "CELLSIZE", 1, 1)

In [ ]:
def las_to_raster(las):
    """This function converts las data to las dataset, and turn the dataset into raster.
    The input should be .las files or folder and the ouput will be raster file"""
    dem = 'DEM_raster'
    arcpy.CreateLasDataset_management(las, 'dataset.lasd', 'RECURSION')  # convert las data into las dataset
    arcpy.LasDatasetToRaster_conversion('dataset.lasd', dem, "ELEVATION", "BINNING AVERAGE NATURAL_NEIGHBOR",
                                        "FLOAT", "CELLSIZE", 1, 1)  # convert las dataset into raster
    return dem

In [3]:
dem = las_to_raster('data/Government Hill/1662_2638.las')

In [ ]:
#Project dem into different coordinate system
out_coordinate_system = arcpy.SpatialReference(26934)
arcpy.ProjectRaster_management('GH', 'XY_GH', out_coordinate_system, "BILINEAR")
#Covert unit of z value from feet to meter
Project_raster = Times('XY_GH', 0.3048)
Project_raster.save('Project_GH')

In [ ]:
def project_raster(dem):
    """This function is used to convert unit of corrdination system from feet to meter.
    The input should be a raster and the output will be projected raster file"""
    raster = 'projected_DEM'
    out_coordinate_system = arcpy.SpatialReference(26934)
    arcpy.ProjectRaster_management(dem, 'XY_dem', out_coordinate_system, "BILINEAR", '1') #Project XY coordination system
    pro_raster = Times('XY_dem', 0.3048) #Covert unit of z value from feet to meter
    pro_raster.save(raster)
    return raster

In [4]:
raster = project_raster(dem)

In [ ]:
#Aspect (3D Analyst or Spatial Analyst)
arcpy.Aspect_3d('Project_GH', 'aspect')
#filter south facing or horizontal aspect. Flat, 112.5 <= aspect <= 247.5, set value to 1, others to None
filter_aspect = Con((Raster('aspect') == -1) | (Raster('aspect') >= 112.5) & (Raster('aspect') <= 247.5), 1, '')
# save aspect_raster
filter_aspect.save("filtered_aspect.tif")

In [ ]:
#Slope (3D Analyst or Spatial Analyst)
arcpy.Slope_3d('Project_GH', 'slope', "DEGREE", 1)
# filter slope degree <= 35 to 1, others to None
filter_slope = Con(Raster('slope') <= 35, 1, '')
#save slope_raster
filter_slope.save("filtered_slope.tif")

In [ ]:
#Combine slope and aspect
result = Times("filtered_aspect.tif", "filtered_slope.tif")
result.save("aspect_slope")

In [ ]:
def create_mask(raster):
    """This function creates a mask to filter unsuitable location to install solar panels.
    The input should be a raster and the output will be mask raster"""
    mask = 'mask_raster'
    arcpy.Aspect_3d(raster, 'aspect')  # filter aspect
    # filter south facing or horizontal aspect. Flat, 112.5 <= aspect <= 247.5, set value to 1, others to None
    filter_aspect = Con((Raster('aspect') == -1) | (Raster('aspect') >= 112.5) & (Raster('aspect') <= 247.5), 1, '')
    filter_aspect.save("filtered_aspect.tif")
    arcpy.Slope_3d(raster, 'slope', "DEGREE", 1)  # filter slope
    # filter slope degree <= 35 to 1, others to None
    filter_slope = Con(Raster('slope') <= 35, 1, '')
    filter_slope.save("filtered_slope.tif")
    result = Times("filtered_aspect.tif", "filtered_slope.tif")  # Combine slope and aspect
    result.save(mask)
    return maskr

In [5]:
mask = create_mask(raster)

In [ ]:
#Combine with raster
result = Times("aspect_slope", "GH")
result.save("result")

In [ ]:
#Extract by building polygon
building_raster = ExtractByMask("result", "data/buildings/buildings.shp")
building_raster.save("braster")

In [ ]:
#Area Solar Radiation (Spatial Analyst)
# unit: watt hours per square meter (WH/m2) ??
solar = AreaSolarRadiation('braster', '', '', TimeWholeYear(2018))
solar.save('solar')

In [ ]:
def solar_radiation(mask, raster, polygon):
    """This function is used to calculate solar radiation of filtered location.
    The input should be mask raster, DEM raster, polygon shape file and the output will be solar energy table"""
    solar = 'solar_raster'
    solar_tab = 'solar_energy'
    result = Times(mask, raster)
    result.save("result")
    polygon_raster = ExtractByMask("result", polygon)  # extract by polygon of buildings or parking lots
    polygon_raster.save("poly_raster")
    sol_radiation = AreaSolarRadiation("poly_raster", time_configuration=TimeWholeYear(2018), out_direct_duration_raster='solar_dur')  # calculate solar radiation
    sol_radiation.save("solar_rad")  # unit WH/m2
    solar_result = Divide('solar_rad', "solar_dur")
    solar_result.save(solar)  # unit W/m2
    ZonalStatisticsAsTable(polygon, 'FID', solar, solar_tab, "DATA", "SUM")  # generate solar energy table
    return solar_tab

In [7]:
solar_tab = solar_radiation(mask, raster, 'data/buildings/buildings.shp')

In [ ]:
#Zonal Statistics
ZonalStatisticsAsTable ('data/buildings/buildings.shp', 'FID', 'solar_raster', 'solar_energy', "DATA", "SUM")

In [ ]:
def join_shape(solar_tab, building, address):
    """This function is used to join the solar energy table with building and address shape files
    The input should be table, building and address shape files, and the output should be joined shape file"""
    arcpy.AddIndex_management(solar_tab, 'FID')  # Index.
    # join table to building
    arcpy.MakeFeatureLayer_management(building, 'build_layer')
    arcpy.AddJoin_management('build_layer', 'FID', solar_tab, 'FID', 'KEEP_COMMON')
    arcpy.CopyFeatures_management('build_layer', 'solar_build')
    # join address to building
    

In [9]:
# Index.
arcpy.AddIndex_management('solar_energy', 'FID')
#Add Join
arcpy.MakeFeatureLayer_management('data/buildings/buildings.shp', 'solar_layer')
arcpy.AddJoin_management('solar_layer', 'FID', 'solar_energy', 'FID', 'KEEP_COMMON')
arcpy.CopyFeatures_management('solar_layer', 'solar_build')

<Result 'data/temporary data\\solar_build.shp'>

In [ ]:
#Spatial Join
arcpy.SpatialJoin_analysis('solar_build', 'data/address_pt/address_pt.shp', 'buil_addr',
                           'JOIN_ONE_TO_ONE', 'KEEP_ALL', '#', 'CLOSEST')

In [ ]:
# create field_mapping
fieldmappings = arcpy.FieldMappings()

fld_st_add = arcpy.FieldMap()
fld_cityzip = arcpy
fld_cou =
fld_area =
fld_solar =

In [ ]:
#Covert to kml
arcpy.MakeFeatureLayer_management('data/temporary data/buil_addr.shp', 'addr_layer')
arcpy.LayerToKML_conversion('addr_layer', 'final.kmz')

In [ ]:
#Extract building polygon
arcpy.RasterToPolygon_conversion('GH', 'building', "NO_SIMPLIFY", "VALUE")

In [ ]:
#Hillshade (3D Analyst or Spatial Analyst)
#Haven't used yet
arcpy.HillShade_3d(raster, hillshaded_raster, 315, 45, "SHADOWS", 1)

In [ ]:
# Replace a layer/table view name with a path to a dataset (which can be a layer file) or create the layer/table view within the script
# The following inputs are layers or table views: "solar_build", "address_pt"
arcpy.SpatialJoin_analysis(target_features="solar_build", join_features="address_pt", out_feature_class="C:/Users/acer/Documents/ArcGIS/Default1.gdb/solar_build_SpatialJoin", join_operation="JOIN_ONE_TO_ONE", join_type="KEEP_ALL", field_mapping='solar_en_4 "solar_en_4" true true false 19 Double 0 0 ,First,#,solar_build,solar_en_4,-1,-1;ST_ADD "ST_ADD" true true false 75 Text 0 0 ,First,#,address_pt,ST_ADD,-1,-1;CITYSTZIP "CITYSTZIP" true true false 75 Text 0 0 ,First,#,address_pt,CITYSTZIP,-1,-1;COUNCIL "COUNCIL" true true false 50 Text 0 0 ,First,#,address_pt,COUNCIL,-1,-1', match_option="CLOSEST", search_radius="", distance_field_name="")